In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [2]:
import os
project_root = os.path.dirname(os.path.abspath(os.getcwd()))
os.chdir(project_root)

In [3]:
# tools : get_emails, parse_emails
from tools.api_tools import get_emails, parse_emails
# schemas : EmailItem, ParsedQuery, EmailFetchInput, EmailFetchOutput, EmailFetchOutputs, UserChoice
from py_files.schemas import EmailItem, ParsedQuery, EmailFetchInput, EmailFetchOutput, UserChoice

In [27]:
import yaml
from datetime import datetime
# prompts
def load_prompts(name : str):
    with open("prompts/prompts.yaml", "r") as f:
        prompts = yaml.safe_load(f)
        
    prompt = prompts[name]

    if name == "fetching_email_prompt":
        today_date = datetime.now().strftime("%Y-%m-%d")
        prompt["instructions"] = prompt["instructions"].format(today_date = today_date)
        
    return prompt


In [ ]:
# state는 잘 모르겠다..
from typing import TypedDict

class EmailAgentState(TypedDict, total = False):
    query : str
    top_k : int
    emails : List[EmailItem]
    fetched_emails : List[EmailFetchOutput]
    selected_ids : List[str]
    # status : Literal["INITIAL", "FETCHED", "AWAIT_USER_CONFIRM", "CONFIRMED", "ERROR"]
    error : Optional[str]

In [5]:
# 이메일 전부 불러오기 - 50개 불러오는 데 23초 정도
email_dicts = parse_emails.invoke({})

Chat messages : 
- subject: 이번 주 미팅 일정 | sender: "박준희" <jhpark0256@naver.com>
| id: 1998aa39fff81eac | Snippet: 혹시 이번 주 중으로 온라인 미팅이 가능할까요? 화요일 오후나 목요일 오전이 편할 것 같습니다.
- subject: 내일까지 해야할 일 안내드립니다. | sender: "박준희" <jhpark0256@naver.com>
| id: 1998a2c2ff890124 | Snippet: 내일까지 Gmail Responder Agent의 기초 틀을 마련해주세요. 최소한 구현해야 하는 기능은 다음과 같습니다: 1. 전체 이메일을 로드 2. 그중 원하는 이메일을 선택 3. 그에 대한 답변을 작성 4. 자동으로 전송 내일까지 구현 바랍니다.
- subject: 안녕하세용 | sender: "박준희" <jhpark0256@naver.com>
| id: 19984a9d489f5afe | Snippet: 이게 정말 되는 걸까?!?!?
- subject: Now shipped: Cursor CLI | sender: Michael from Cursor <michael@mail.cursor.com>
| id: 1998420b87172291 | Snippet: Cursor CLI is now available - code from anywhere ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏﻿ ‌​‍‎‏
- subject: [Notice] Enjoy new benefits with additional pricing options from October 1 | sender: Upstage Console <console@upstage.ai>
| id: 19983a881d374778 

In [6]:
email_dicts[0]

{'id': '1998aa39fff81eac',
 'subject': '이번 주 미팅 일정',
 'sender': '"박준희" <jhpark0256@naver.com>',
 'date': 'Sat, 27 Sep 2025 19:06:40 +0900',
 'email_content': '혹시 이번 주 중으로 온라인 미팅이 가능할까요?\n화요일 오후나 목요일 오전이 편할 것 같습니다. \n'}

In [7]:
def parse_query(query : str) -> ParsedQuery:
    """쿼리를 파싱하여 주어진 형식에 맞는 파싱 결과를 반환"""
    from langchain_core.prompts import ChatPromptTemplate
    from langchain.chat_models import init_chat_model

    query_parser_prompt = load_prompts("query_parser_prompt")
    
    llm_structured = init_chat_model("openai:gpt-4o-mini", temperature=0.0).with_structured_output(ParsedQuery)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", query_parser_prompt['role'] + "\n" + query_parser_prompt['instructions']),
            ("user", query_parser_prompt["inputs"].format(query = query))
        ]
    )
    messages = prompt.format_messages(query = query)
    response = llm_structured.invoke(messages)

    return response

In [8]:
# 1분 정도 소요
query = "어제 왔던 미팅 관련 메일 찾아줘."
parsed_query = parse_query(query)
parsed_query

ParsedQuery(email_content=['어제 왔던 미팅 관련 메일'], when='어제', sender=None)

In [ ]:
from typing import List, Dict, Optional, Literal

def fetch_email(parsed_query: ParsedQuery, email_dicts: List[dict]) -> EmailFetchOutput:
    """파싱된 쿼리를 사용하여 관련 이메일 찾아서 반환 + 수동 메시지 생성"""
    from langchain.chat_models import init_chat_model
    from langchain_core.messages import SystemMessage, HumanMessage

    fetching_email_prompt = load_prompts("fetching_email_prompt")
    
    # 이메일 포맷팅
    formatted_emails = []
    for email in email_dicts:
        formatted_email = {
            "id": email["id"],
            "subject": email["subject"], 
            "sender": email["sender"],
            "date": email["date"],
            "content": email["content"][:200] + "..." if len(email["content"]) > 200 else email["content"]
        }
        formatted_emails.append(formatted_email)
    
    # 메시지 수동 생성
    system_content = fetching_email_prompt['role'] + "\n" + fetching_email_prompt['instructions']
    user_content = fetching_email_prompt["inputs"].format(
        parsed_query=parsed_query.model_dump(), 
        emails=formatted_emails
    )
    
    messages = [
        SystemMessage(content=system_content),
        HumanMessage(content=user_content)
    ]
    
    # LLM 호출
    llm_structured = init_chat_model("openai:gpt-4o-mini", temperature=0.0).with_structured_output(EmailFetchOutput)
    response = llm_structured.invoke(messages)

    return response

In [30]:
fetched_email = fetch_email(parsed_query, email_dicts)
print("Fetched Email:", fetched_email)

Fetched Email: id='1998aa39fff81eac' subject='이번 주 미팅 일정' sender='"박준희" <jhpark0256@naver.com>' date='Sat, 27 Sep 2025 19:06:40 +0900' email_content='혹시 이번 주 중으로 온라인 미팅이 가능할까요?\n화요일 오후나 목요일 오전이 편할 것 같습니다. \n' reasoning="이 이메일은 어제(2025년 9월 27일) 보낸 미팅 관련 메일로, 요청한 내용과 가장 관련이 깊습니다. '어제'라는 시간적 제약과 '미팅'이라는 키워드가 포함되어 있어, 요청한 내용과 일치합니다."


In [ ]:
from langgraph.types import Command